# Cutin Synthase Stabilization Analysis
## Example Workflow Notebook

This notebook demonstrates the complete workflow for analyzing and improving cutin synthase thermostability.

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Structure Preparation

In [ ]:
from structure_preparation.prepare_structure import StructurePreparator

# Load AlphaFold3 structure
pdb_file = '../data/raw/cutin_synthase.pdb'

# Check if file exists
if Path(pdb_file).exists():
    prep = StructurePreparator(pdb_file)
    structure = prep.load_structure()
    
    # Get statistics
    stats = prep.analyze_structure()
    print("Structure Statistics:")
    for key, value in stats.items():
        print(f"  {key}: {value}")
    
    # Get sequence
    sequence = prep.get_sequence()
    print(f"\nSequence length: {len(sequence)} residues")
    print(f"Sequence: {sequence[:50]}...")
    
    # Clean structure
    prep.clean_structure('../data/processed/cutin_synthase_clean.pdb')
else:
    print(f"Structure file not found: {pdb_file}")
    print("Please place your AlphaFold3 structure in data/raw/cutin_synthase.pdb")

## 2. Mutation Screening

In [ ]:
from mutation_screening.mutation_screener import MutationScreener, ThermostabilityPredictor

# Initialize screener
if 'structure' in locals():
    screener = MutationScreener(structure)
    
    # Identify flexible regions
    flexible = screener.identify_flexible_regions()
    print(f"Found {len(flexible)} flexible residues")
    
    if flexible:
        df_flexible = pd.DataFrame(flexible)
        display(df_flexible.head(10))
    
    # Get stabilization strategies
    strategies = screener.suggest_stabilizing_mutations()
    print("\nStabilization Strategies:")
    for strategy, details in strategies.items():
        print(f"\n{strategy}:")
        print(f"  {details['description']}")
else:
    print("Load structure first (run previous cell)")

## 3. Generate and Rank Mutations

In [ ]:
# Generate mutation library for flexible positions
if 'flexible' in locals() and flexible:
    # Take top 5 most flexible positions
    positions = [f"{res['resid']}" for res in flexible[:5]]
    
    # Generate mutations
    mutations = screener.generate_mutation_library(positions)
    
    # Predict and rank
    predictor = ThermostabilityPredictor()
    ranked_mutations = predictor.rank_mutations(mutations[:20])  # Test first 20
    
    # Display results
    df_mutations = pd.DataFrame(ranked_mutations)
    print("Top 10 Stabilizing Mutations (most negative ΔΔG):")
    display(df_mutations.head(10))
    
    # Plot ΔΔG distribution
    plt.figure(figsize=(10, 6))
    plt.hist(df_mutations['ddg'], bins=20, edgecolor='black')
    plt.axvline(x=0, color='red', linestyle='--', label='ΔΔG = 0')
    plt.xlabel('ΔΔG (kcal/mol)')
    plt.ylabel('Count')
    plt.title('Distribution of Predicted ΔΔG Values')
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No flexible residues found. Run mutation screening first.")

## 4. MD Simulation Setup

In [ ]:
from md_simulation.setup_md import MDSimulationSetup

# Setup simulation for cleaned structure
clean_pdb = '../data/processed/cutin_synthase_clean.pdb'

if Path(clean_pdb).exists():
    md_setup = MDSimulationSetup(clean_pdb)
    
    # Display simulation parameters
    print("Simulation Parameters:")
    for key, value in md_setup.simulation_params.items():
        print(f"  {key}: {value}")
    
    # Generate MDP files
    mdp_files = md_setup.generate_mdp_files('../data/processed/mdp_files')
    print("\nGenerated MDP files:")
    for key, path in mdp_files.items():
        print(f"  {key}: {path}")
else:
    print(f"Clean structure not found: {clean_pdb}")

## 5. Trajectory Analysis (Example)

In [ ]:
from analysis.analyze_results import TrajectoryAnalyzer

# Initialize analyzer (will work when trajectory files are available)
analyzer = TrajectoryAnalyzer()

# Generate example RMSD data
rmsd_data = analyzer.calculate_rmsd()

# Plot RMSD
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(rmsd_data['time'], rmsd_data['rmsd'], linewidth=0.5)
plt.xlabel('Time (ns)')
plt.ylabel('RMSD (Å)')
plt.title('Backbone RMSD over Time')
plt.grid(True, alpha=0.3)

# Generate example Rg data
rg_data = analyzer.calculate_radius_of_gyration()

plt.subplot(1, 2, 2)
plt.plot(rg_data['time'], rg_data['rg'], linewidth=0.5, color='orange')
plt.xlabel('Time (ns)')
plt.ylabel('Radius of Gyration (Å)')
plt.title('Radius of Gyration over Time')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Note: These are example plots with simulated data.")
print("Replace with actual trajectory files for real analysis.")

## 6. Protein Descriptor Analysis

In [ ]:
from analysis.analyze_results import ProteinDescriptorCalculator

# Calculate descriptors
if 'structure' in locals():
    calc = ProteinDescriptorCalculator(structure)
    descriptors = calc.calculate_thermostability_indicators()
    
    print("Thermostability Descriptors:")
    for key, value in descriptors.items():
        print(f"  {key}: {value}")
    
    catalytic = calc.calculate_catalytic_descriptors()
    print("\nCatalytic Descriptors:")
    for key, value in catalytic.items():
        print(f"  {key}: {value}")
else:
    print("Load structure first")

## 7. Variant Comparison (Example)

In [ ]:
# Example comparison of multiple variants
variants = {
    'Wild Type': {
        'aliphatic_index': 85.0,
        'instability_index': 38.5,
        'gravy': -0.2,
        'salt_bridges': 12,
        'avg_rmsd': 2.5
    },
    'Mutant A': {
        'aliphatic_index': 92.0,
        'instability_index': 35.2,
        'gravy': -0.1,
        'salt_bridges': 15,
        'avg_rmsd': 2.1
    },
    'Mutant B': {
        'aliphatic_index': 88.0,
        'instability_index': 36.8,
        'gravy': -0.15,
        'salt_bridges': 14,
        'avg_rmsd': 2.3
    }
}

df_variants = pd.DataFrame(variants).T
print("Variant Comparison:")
display(df_variants)

# Radar plot comparison
from math import pi

categories = ['Aliphatic\nIndex', 'Salt\nBridges', 'Stability\n(low RMSD)']
N = len(categories)

# Normalize data for radar plot
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))

for variant, data in variants.items():
    values = [
        data['aliphatic_index'] / 100,
        data['salt_bridges'] / 20,
        (5 - data['avg_rmsd']) / 5  # Inverse RMSD (higher is better)
    ]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=variant)
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1)
ax.set_title('Variant Comparison (Normalized)', size=16, y=1.08)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.show()

print("\nNote: This is example data. Replace with actual variant measurements.")

## 8. Summary and Recommendations

In [ ]:
print("="*70)
print("CUTIN SYNTHASE STABILIZATION PROJECT - SUMMARY")
print("="*70)

print("\n✓ Completed Steps:")
print("  1. Structure preparation and validation")
print("  2. Mutation screening and ranking")
print("  3. MD simulation setup")
print("  4. Analysis framework established")

print("\n→ Next Steps:")
print("  1. Run MD simulations for top mutation candidates")
print("  2. Compare stability and activity metrics")
print("  3. Select 2-3 best variants for experimental validation")
print("  4. Document results in final report")

print("\n📊 Key Metrics to Monitor:")
print("  - RMSD < 3 Å (structural stability)")
print("  - Aliphatic index > 85 (thermostability)")
print("  - Instability index < 40 (overall stability)")
print("  - Increased salt bridges (electrostatic stability)")
print("  - Maintained active site geometry (catalytic activity)")

print("\n" + "="*70)